In [13]:
from math import pi

import numpy as np
from scipy.linalg import expm
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Statevector

In [14]:
def qft_gate(nbdigits):
    qr = QuantumRegister(nbdigits, 'qr')
    circuit = QuantumCircuit(qr)
    for i in range(nbdigits):
        circuit.h(qr[i])
        for j in range(i+1, nbdigits):
            circuit.cp(2*pi/2**(j-i+1), j, i)
    for i in range(nbdigits // 2):
        circuit.swap(i, nbdigits-i-1)
    return circuit.to_gate()


def inv_qft_gate(nbdigits):
    qr = QuantumRegister(nbdigits, 'qr')
    circuit = QuantumCircuit(qr)
    for i in range(nbdigits // 2):
        circuit.swap(i, nbdigits-i-1)
    for i in range(nbdigits):
        circuit.h(qr[i])
        for j in range(i+1, nbdigits):
            rot_phase = - 2*pi/2**(j-i+1)
            circuit.cp(rot_phase, j, i)
    return circuit.to_gate()


def phase_estimation_gate(gate, nbdigits, teststate_nbdigits):
    # customized controlled gate
    controlled_gate = gate.control(1)

    # preparing for the circuit
    circuit = QuantumCircuit(nbdigits + teststate_nbdigits)
    circuit.h(range(nbdigits))
    for i in range(nbdigits):
        for _ in range(2 ** (i)):
            circuit.append(controlled_gate, [i] + [j + nbdigits for j in range(teststate_nbdigits)])
    circuit.append(inv_qft_gate(nbdigits), range(nbdigits))

    return circuit.to_gate()


def inv_phase_estimation_gate(gate, nbdigits, teststate_nbdigits):
    # customized controlled gate
    controlled_gate = gate.control(1)

    # preparing for the circuit
    circuit = QuantumCircuit(nbdigits + teststate_nbdigits)
    circuit.append(qft_gate(nbdigits), range(nbdigits))
    for i in range(nbdigits+1, 0, -1):
        for _ in range(2 ** (i)):
            circuit.append(controlled_gate, [i] + [j + nbdigits for j in range(teststate_nbdigits)])
    circuit.h(range(nbdigits))

    return circuit.to_gate()


In [15]:
nbinputdigits = 1
nbclockdigits = 1
nbancilladigits = 1

In [16]:
A = np.array([[1., 2.], [2., -3.]])
b = np.array([4., -1.])
b = b / np.linalg.norm(b)    # normalizing it

In [17]:
initial_state = np.zeros(2**(nbancilladigits+nbclockdigits+nbinputdigits), dtype=np.complex_)
initial_state[2**(nbancilladigits+nbclockdigits)] = b[0]
initial_state[2**(nbancilladigits+nbclockdigits) + 1] = b[1]

In [18]:
gate = UnitaryGate(expm(2*pi*(1.j)*A))

In [19]:
circuit = QuantumCircuit(nbancilladigits + nbclockdigits + nbinputdigits, nbancilladigits)
circuit.initialize(initial_state)
circuit.append(phase_estimation_gate(gate, nbclockdigits, nbinputdigits), range(nbancilladigits, nbancilladigits+nbclockdigits+nbinputdigits))


In [20]:
Statevector(circuit).probabilities_dict()

{'000': 0.09131820408368199,
 '001': 0.005707387755230128,
 '010': 0.09131820408368192,
 '011': 0.005707387755230122,
 '100': 0.601891227769431,
 '101': 0.03761820173558946,
 '110': 0.15664883465143822,
 '111': 0.009790552165714898}